In [1]:
import pandas as pd
import numpy as np

In [2]:
def f(x):
    return np.exp(x)


def count_c(y, h, n):
    top = np.ones(n)
    top[0] = 0
    low = np.ones(n)
    low[-1] = 0
    mid = np.empty(n)
    mid.fill(4)

    res = np.empty((n,))
    for i in range(n):
        res[i] = 3/h**2 *(y[i] - 2*y[i+1] + y[i+2])

    alpha = np.zeros((n,))
    beta = np.zeros((n,))

    alpha[0] = -low[0] / mid[0]
    beta[0] = res[0] / mid[0]

    for i in range(1, n):
        alpha[i] = -low[i]/(top[i]*alpha[i-1] + mid[i])
        beta[i] = (res[i] - top[i]*beta[i-1])/(top[i]*alpha[i-1] + mid[i])

    x = np.zeros((n,))
    x[n-1] = beta[n - 1]

    for i in range(n-1,0,-1):
        x[i-1] = alpha[i-1]*x[i] + beta[i-1]

    return list(x)


def find_coefs(y,c,h,n):
    a = []
    b = []
    d = []
    for i in range(1,n+1):
        a.append(
            y[i-1]
        )
        b.append(
            (y[i] - y[i-1])/h - h/3*(c[i] + 2*c[i-1])
        )
        d.append(
            (c[i] - c[i-1])/(3*h)
        )
    a.append(0)
    b.append(0)
    d.append(0)
    return a,b,d

In [4]:
left = 0
right = 3
n = 5
h = (right-left)/n

x = np.linspace(left,right, num=n+1,endpoint=True)
y = np.vectorize(f)(x)
c = count_c(y, h, n-1)
c = [0] + c + [0]
a,b,d = find_coefs(y=y, c=c, h=h, n=n)

expected_results = []
spline_results = []
points = [left + point_num * h/2 for point_num in range(2*n+1)]

for point_num in range(2*n+1):
    point = points[point_num]
    i = point_num//2
    if point_num == 2*n:
        i -= 1

    spline_result = a[i] + \
               b[i] * (point - x[i]) + \
               c[i] * (point - x[i]) ** 2 + \
               d[i] * (point - x[i]) ** 3
    spline_results.append(spline_result)
    expected_results.append(f(point))

expected_results = np.array(expected_results)
spline_results = np.array(spline_results)

df = pd.DataFrame(
    {'x':points,
        'expected': expected_results,
     'splined': spline_results,
     'delta': np.abs(expected_results-spline_results)}
)
print(df)

      x   expected    splined         delta
0   0.0   1.000000   1.000000  0.000000e+00
1   0.3   1.349859   1.367525  1.766575e-02
2   0.6   1.822119   1.822119  0.000000e+00
3   0.9   2.459603   2.448268  1.133547e-02
4   1.2   3.320117   3.320117  0.000000e+00
5   1.5   4.481689   4.504539  2.284946e-02
6   1.8   6.049647   6.049647  0.000000e+00
7   2.1   8.166170   8.077313  8.885716e-02
8   2.4  11.023176  11.023176  0.000000e+00
9   2.7  14.879732  15.196286  3.165540e-01
10  3.0  20.085537  20.085537  3.552714e-15
